# Chapter02: Foundation of OpenAI's Chat API

In [4]:
import os
from dotenv import load_dotenv
from openai import OpenAI


loaded = load_dotenv()
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

## 2.1 Use `tiktoken` to Count the Number of Tokens

In [22]:
import tiktoken

text = "Can you tell me what should I do ?"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))


9


## 2.2 Simple Q&A

In [23]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, I am Chen!"}
    ],
)

# `indent=2` means that each level of nesting is indented by two spaces.
print(response.to_json(indent=2))

{
  "id": "chatcmpl-BvgsHOT84gp9vaN4yMbTku8OqV8n6",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello, Chen! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1753089205,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 11,
    "prompt_tokens": 23,
    "total_tokens": 34,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


### Obtain cline's response directly

In [24]:
content = response.choices[0].message.content
print(content)

Hello, Chen! How can I assist you today?


### OpenAI's ChatAPI is stateless. Hence, the conversation history is not saved, we need to include it with each request.

In [11]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, I am Chen!"},
        {"role": "assistant", "content": "Hello, Chen! How can I assist you today?"},
        {"role": "user", "content": "Do you know what my name is?"}
    ],
)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-BvgSf7kbbfun7YlqNxlxAKq5FMFT3",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Yes, you mentioned that your name is Chen. How can I help you, Chen?",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1753087617,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 50,
    "total_tokens": 68,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


## 2.3 Streaming Mode

When using **streaming mode**, the model's response are split into individual chunks and the original `message` field becomes `delta`.

This creates the effect of text appearing gradually, just like in a real-time conversation.

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I am Chen. Can you say something with 50 words? You should break a line when a sentence ends."},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        # `end=""` overrides the default newline suffix (which is "\n"), so nothing is appended after each `print` call.
        # `flush=True` forces Python to immediately flush its output buffer to the terminal, 
        # ensuring each chunk is displayed right away rather than being buffered.
        print(content, end="", flush=True)

Hello, Chen!  
It's great to meet you.  
Life is full of opportunities  
and experiences waiting to be embraced.  
Embrace each moment with joy  
and curiosity, as they shape who you are.  
Every encounter teaches a lesson,  
and every challenge brings growth.  
Keep exploring, learning, and shining brightly!

## 2.4 Json Mode

In [31]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "Please output the appeared persons in Json format. \n{'people': ['aaa', 'bbb']}",
        },
        {
            "role": "user",
            "content": "Ben is talking with Jenny. In the past, grandfather played chess with grandmother in a building."
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

{
  "people": ["Ben", "Jenny", "grandfather", "grandmother"]
}


## 2.5 Image Input

In [33]:
import base64

def jpg_to_data_uri(path: str) -> str:
    '''
    This function is used to convert *.jpg to urls which can be feed into GPT-4o.
    '''
    with open(path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("ascii")
    return f"data:image/jpeg;base64,{b64}"

image_path = "example_img.jpeg"
image_uri = jpg_to_data_uri(image_path)

In [37]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Please describe this image."},
                {"type": "image_url", "image_url": {"url": image_uri}}
            ],
        }
    ],
)

print(response.choices[0].message.content)

The image shows a table filled with various dishes, likely from an Asian cuisine. Visible on the table are:

1. Grilled eggplant topped with sauce.
2. Dishes of clams with sauce and vegetables.
3. Grilled oysters with a savory topping.
4. A plate of stir-fried noodles or shredded vegetables with chili.
5. Skewers of grilled fish.
6. Another plate of seafood with sauce.

There's also a can of soda and some people partially visible in the background. The spread appears to be a mix of seafood and vegetable dishes.


## 2.6 Function Calling

Functions need to be called locally.

So we should send a request asking which functions should be utilized and call it locally.

In [31]:
import json

def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

**Prepare a tool list indicating GPT-4o the available funcions.**

In [32]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location":{
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            }
        }
    }
]

**Now we know we should use the `get_current_weather` function.**

In [33]:
messages = [
    {"role": "user", "content": "How about Tokyo's weather?"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-BvjnEx5SqpRAOVDyTZw5mUWcL20bp",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "tool_calls": [
          {
            "id": "call_IkzO4U0syA3dU5noC8h5JJP1",
            "function": {
              "arguments": "{\"location\":\"Tokyo\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1753100424,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 78,
    "total_tokens": 93,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_t

In [34]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "How about Tokyo's weather?"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "annotations": [],
    "tool_calls": [
      {
        "id": "call_IkzO4U0syA3dU5noC8h5JJP1",
        "function": {
          "arguments": "{\"location\":\"Tokyo\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  }
]


**Call the `get_current_weather` locally**

In [35]:
available_functions = {
    "get_current_weather": get_current_weather
}

# Iterate over funcions that agent want to utilize
for tool_call in response_message.tool_calls:
    # Obtain the function name
    function_name = tool_call.function.name
    # Obtain the corresponding functhon object
    function_to_call = available_functions[function_name]
    # Load the parameters
    function_args = json.loads(tool_call.function.arguments)
    # Obtain and pass the parameters
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )

    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "How about Tokyo's weather?"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "annotations": [],
    "tool_calls": [
      {
        "id": "call_IkzO4U0syA3dU5noC8h5JJP1",
        "function": {
          "arguments": "{\"location\":\"Tokyo\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_IkzO4U0syA3dU5noC8h5JJP1",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Tokyo\", \"temperature\": \"10\", \"unit\": null}"
  }
]


In [36]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))
print("-------------------------------------------------------")
print(second_response.choices[0].message.content)

{
  "id": "chatcmpl-BvjnOaNOjTHvovcfNJScZuW2fti3v",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The current temperature in Tokyo is 10 degrees.",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1753100434,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_a288987b44",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 54,
    "total_tokens": 64,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
-------------------------------------------------------
The current temperature in Tokyo is 10 degrees.
